In [ ]:
# %load_ext kedro.extras.extensions.ipython
# %reload_kedro /home/ofurman/Probabilistic-Flow-Boosting

In [2]:
import sys
sys.path.append("../src/")

import os
import pandas as pd
from probabilistic_flow_boosting.models.nodeflow import NodeFlow, NodeFlowDataModule
from probabilistic_flow_boosting.models.node_gmm import NodeGMM
from probabilistic_flow_boosting.extras.datasets.uci_dataset import UCIDataSet
from probabilistic_flow_boosting.pipelines.modeling.nodes.nodeflow import train_nodeflow

/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/lightning_fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning_fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/pytorch_lightning/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pytorch_lightning')`.
Implementing implicit namespace packages (as 

In [3]:
x_train = UCIDataSet(
    filepath_data="../data/01_raw/UCI/energy/data.txt",
    filepath_index_columns="../data/01_raw/UCI/energy/index_features.txt",
    filepath_index_rows="../data/01_raw/UCI/energy/index_train_0.txt"
).load()
y_train = UCIDataSet(
    filepath_data="../data/01_raw/UCI/energy/data.txt",
    filepath_index_columns="../data/01_raw/UCI/energy/index_target.txt",
    filepath_index_rows="../data/01_raw/UCI/energy/index_train_0.txt"
).load()

x_test = UCIDataSet(
    filepath_data="../data/01_raw/UCI/energy/data.txt",
    filepath_index_columns="../data/01_raw/UCI/energy/index_features.txt",
    filepath_index_rows="../data/01_raw/UCI/energy/index_test_0.txt"
).load()
y_test = UCIDataSet(
    filepath_data="../data/01_raw/UCI/energy/data.txt",
    filepath_index_columns="../data/01_raw/UCI/energy/index_target.txt",
    filepath_index_rows="../data/01_raw/UCI/energy/index_test_0.txt"
).load()

In [4]:
datamodule = NodeFlowDataModule(x_train, y_train, x_test, y_test, split_size=0.8, batch_size=2048)

model_filepath = "../data/06_models/UCI/energy/model_0"
NodeFlow.load(model_filepath)

NodeFlow(
  (tree_model): DenseODSTBlock(
    (0): ODST(in_features=8, num_trees=200, depth=4, tree_dim=2, flatten_output=True)
    (1): ODST(in_features=408, num_trees=200, depth=4, tree_dim=2, flatten_output=True)
    (2): ODST(in_features=808, num_trees=200, depth=4, tree_dim=2, flatten_output=True)
    (3): ODST(in_features=1208, num_trees=200, depth=4, tree_dim=2, flatten_output=True)
  )
  (flow_model): ContinuousNormalizingFlow(
    (flow): SequentialFlow(
      (chain): ModuleList(
        (0): MovingBatchNorm1d(1, eps=0.0001, decay=0.1, bn_lag=0.0, affine=True)
        (1): CNF(
          (odefunc): ODEfunc(
            (diffeq): ODEnet(
              (layers): ModuleList(
                (0): ConcatSquashLinear(
                  (_layer): Linear(in_features=1, out_features=4, bias=True)
                  (_hyper_bias): Linear(in_features=1601, out_features=4, bias=False)
                  (_hyper_gate): Linear(in_features=1601, out_features=4, bias=True)
                )
  

In [5]:
model_params = {
        "num_layers": 4,
        "depth": 4,
        "tree_output_dim": 2,
        "num_trees": 200,
        "flow_hidden_dims": [4,4]
}

best_model_path = train_nodeflow(
    x_train=x_train,
    y_train=y_train,
    n_epochs=400,
    patience=40,
    split_size=0.8,
    batch_size=2048,
    model_hyperparams=model_params
)
model = NodeFlow.load_from_checkpoint(best_model_path, input_dim=x_train.shape[1], output_dim=y_train.shape[1], **model_params)
os.remove(best_model_path)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/ofurman/Probabilistic-Flow-Boosting/notebooks/lightning_logs
/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (552). n_quantiles is set to n_samples.
  warnings.warn(
LOCAL_RAN

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ofurman/Probabilistic-Flow-Boosting/notebooks/../src/probabilistic_flow_boosting/models/node/odst.py:143: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn(


/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:281: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 8: 100%|██████████| 1/1 [00:02<00:00,  2.20s/it, v_num=0, val_nll=3.290, train_nll=3.340]

/home/ofurman/Probabilistic-Flow-Boosting/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
